# **SDG Prediction**

## **Dependencies**

In [1]:
from transformers import pipeline
import pandas as pd
from tqdm import tqdm

## **SDG Classifier**

### Load Model

Model predicts first 15 sdg

https://huggingface.co/jonas/sdg_classifier_osdg

In [2]:
pipe = pipeline("text-classification", model="jonas/bert-base-uncased-finetuned-sdg")

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

c:\Users\UserNA6153\anaconda3\envs\pamb2\lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\UserNA6153\.cache\huggingface\hub\models--jonas--bert-base-uncased-finetuned-sdg. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/348 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

### Load CSV

In [3]:
df = pd.read_csv("../../../src/merged_orgas.csv")
df.head(1)

,iati_id,iati_orga_id,orga_abbreviation,orga_full_name,client,title_en,title_other,title_main,organization,country_code_list,...,actual_end,last_update,crs_5_code,crs_5_name,crs_3_code,crs_3_name,docs,title_and_description,sgd_pred_code,sgd_pred_str
0,DE-1-201822287-0,DE-1,bmz,Bundesministerium für wirtschaftliche Zusammen...,BMZ,Strengthening quality infrastructure for trade...,Stärkung der Qualitätsinfrastruktur für den Ha...,Strengthening quality infrastructure for trade...,Bundesministerium für wirtschaftliche Zusammen...,NaN,...,2016-03-14T00:00:00Z,2024-02-29T00:00:00Z,33130;,Regional trade agreements (RTAs);,331;,Trade Policies & Regulations;,NaN,Strengthening quality infrastructure for trade...,9,"8 9. Build resilient infrastructure, promot..."


In [4]:
df.columns

Index(['iati_id', 'iati_orga_id', 'orga_abbreviation', 'orga_full_name',
       'client', 'title_en', 'title_other', 'title_main', 'organization',
       'country_code_list', 'country', 'country_name', 'country_flag',
       'region', 'location', 'description_en', 'description_other',
       'description_main', 'status', 'planned_start', 'actual_start',
       'planned_end', 'actual_end', 'last_update', 'crs_5_code', 'crs_5_name',
       'crs_3_code', 'crs_3_name', 'docs', 'title_and_description',
       'sgd_pred_code', 'sgd_pred_str'],
      dtype='object')

### Load SDG CSV

In [6]:
sdg_df = pd.read_csv("../../../src/codelists/sdg_goals.csv")
sdg_df.head(16)

,code,name,description,language,category,category-name,category-description
0,1,1. End poverty in all its forms everywhere,NaN,en,NaN,NaN,NaN
1,2,"2. End hunger, achieve food security and impro...",NaN,en,NaN,NaN,NaN
2,3,3. Ensure healthy lives and promote well-being...,NaN,en,NaN,NaN,NaN
3,4,4. Ensure inclusive and equitable quality educ...,NaN,en,NaN,NaN,NaN
4,5,5. Achieve gender equality and empower all wom...,NaN,en,NaN,NaN,NaN
5,6,6. Ensure availability and sustainable managem...,NaN,en,NaN,NaN,NaN
6,7,"7. Ensure access to affordable, reliable, sust...",NaN,en,NaN,NaN,NaN
7,8,"8. Promote sustained, inclusive and sustainabl...",NaN,en,NaN,NaN,NaN
8,9,"9. Build resilient infrastructure, promote inc...",NaN,en,NaN,NaN,NaN
9,10,10. Reduce inequality within and among countries,NaN,en,NaN,NaN,NaN


### Apply Model

In [8]:
# Define sdg columns 
df["sgd_pred_code"] = "NaN"
df["sgd_pred_str"] = "NaN"

len_df = len(df)

for index, row in tqdm(df.iterrows(), total=len_df, desc="Processing"):
    if index % 500 == 0:
        print(f" Debugger: {index} / {len_df}")
    descr_row = row['description_main']
    try:
        # nan in pandas is type float
        # check if nan 
            if isinstance(descr_row, float):
                df["sgd_pred_code"][index] = "NaN"
                df["sgd_pred_str"][index] = "NaN"
            else:
                if len(descr_row) > 512:
                    descr_row = descr_row[:512]
                # use clf with description and predict sgd 
                pred = pipe(descr_row)
                pred_str = pred[0]["label"]
                pred_int = int(pred_str)

                # Map sgd codes to names
                sdg_translation = sdg_df.loc[sdg_df['code'] == pred_int, 'name']

                df["sgd_pred_code"][index] = pred_int
                df["sgd_pred_str"][index] = sdg_translation
    except Exception as e:
        print(f"Error {e}: {descr_row}")

df.head()

Processing:   0%|          | 1/27397 [00:00<1:20:00,  5.71it/s]

 Debugger: 0 / 27397


Processing:   2%|▏         | 499/27397 [01:04<43:24, 10.33it/s]  

 Debugger: 500 / 27397


Processing:   4%|▎         | 1002/27397 [02:16<44:11,  9.95it/s] 

...

 Debugger: 26500 / 27397


Processing:  99%|█████████▊| 27000/27397 [1:21:01<01:15,  5.26it/s]

 Debugger: 27000 / 27397


Processing: 100%|██████████| 27397/27397 [1:22:03<00:00,  5.56it/s]


,iati_id,iati_orga_id,orga_abbreviation,orga_full_name,client,title_en,title_other,title_main,organization,country_code_list,...,actual_end,last_update,crs_5_code,crs_5_name,crs_3_code,crs_3_name,docs,title_and_description,sgd_pred_code,sgd_pred_str
0,DE-1-201822287-0,DE-1,bmz,Bundesministerium für wirtschaftliche Zusammen...,BMZ,Strengthening quality infrastructure for trade...,Stärkung der Qualitätsinfrastruktur für den Ha...,Strengthening quality infrastructure for trade...,Bundesministerium für wirtschaftliche Zusammen...,NaN,...,2016-03-14T00:00:00Z,2024-02-29T00:00:00Z,33130;,Regional trade agreements (RTAs);,331;,Trade Policies & Regulations;,NaN,Strengthening quality infrastructure for trade...,9,"8 9. Build resilient infrastructure, promot..."
1,DE-1-201920016-0,DE-1,bmz,Bundesministerium für wirtschaftliche Zusammen...,BMZ,Strengthening of Metrology for the Improvement...,Stärkung des Messwesens in Ägypten zur Verbess...,Strengthening of Metrology for the Improvement...,Bundesministerium für wirtschaftliche Zusammen...,['AG'],...,2016-03-14T00:00:00Z,2024-02-29T00:00:00Z,14010;,Water sector policy and administrative managem...,140;,Water Supply & Sanitation;,NaN,Strengthening of Metrology for the Improvement...,9,"8 9. Build resilient infrastructure, promot..."
2,DE-1-201721877-0,DE-1,bmz,Bundesministerium für wirtschaftliche Zusammen...,BMZ,Strengthening regional integration and coopera...,Stärkung der regionalen Integration und Zusamm...,Strengthening regional integration and coopera...,Bundesministerium für wirtschaftliche Zusammen...,NaN,...,2016-03-14T00:00:00Z,2024-02-29T00:00:00Z,33130;,Regional trade agreements (RTAs);,331;,Trade Policies & Regulations;,NaN,Strengthening regional integration and coopera...,9,"8 9. Build resilient infrastructure, promot..."
3,DE-1-201276351-0,DE-1,bmz,Bundesministerium für wirtschaftliche Zusammen...,BMZ,Strengthening Non-Violent Popular Movements in...,Kapazitätsentwicklung für gewaltfreie Basisbew...,Strengthening Non-Violent Popular Movements in...,Bundesministerium für wirtschaftliche Zusammen...,['VU'],...,2016-03-14T00:00:00Z,2024-03-20T00:00:00Z,15160;,Human rights;,151;,Government & Civil Society-general;,NaN,Strengthening Non-Violent Popular Movements in...,16,15 16. Promote peaceful and inclusive socie...
4,DE-1-201676584-0,DE-1,bmz,Bundesministerium für wirtschaftliche Zusammen...,BMZ,Rebuilding Further Arts after cyclone,Wiederaufbau von Further Arts nach Wirbelsturm,Rebuilding Further Arts after cyclone,Bundesministerium für wirtschaftliche Zusammen...,['VU'],...,2016-03-14T00:00:00Z,2024-03-20T00:00:00Z,73010;,Immediate post-emergency reconstruction and re...,730;,Reconstruction Relief & Rehabilitation;,NaN,Rebuilding Further Arts after cyclone. Rebuild...,11,10 11. Make cities and human settlements in...


### With Batch processing

>> Not faster!

In [ ]:
df["sgd_pred_code"] = "NaN"
df["sgd_pred_str"] = "NaN"

batch_size = 8
n_batches = len(df) // batch_size + (len(df) % batch_size > 0)

for batch_n in tqdm(range(n_batches), desc="Processing batches"):
    batch_start = batch_n * batch_size
    batch_end = (batch_n + 1) * batch_size
    df_batch = df.iloc[batch_start:batch_end]

    for index, row in df_batch.iterrows():
        descr_row = row['description_main']
        try:
            if isinstance(descr_row, float):  
                continue  
            else:
                if len(descr_row) > 512:
                    descr_row = descr_row[:512]
                pred = pipe(descr_row)  
                pred_str = pred[0]["label"]
                pred_int = int(pred_str)
                
                sdg_translation = sdg_df.loc[sdg_df['code'] == pred_int, 'name'].values[0] if not sdg_df.loc[sdg_df['code'] == pred_int, 'name'].empty else "NaN"

                df.loc[index, "sgd_pred_code"] = pred_int
                df.loc[index, "sgd_pred_str"] = sdg_translation
        except Exception as e:
            print(f"Error at index {index}: {e}")

    if (batch_n + 1) % 1 == 0 or batch_n == n_batches - 1:
        tqdm.write(f"Processed batch {batch_n + 1}/{n_batches}")

In [11]:
df["sgd_pred_code"].value_counts()

sgd_pred_code
8      3638
2      3071
11     2981
9      2313
16     2163
3      2134
4      2126
7      1588
1      1547
6      1542
13     1398
5      1344
15      822
12      287
14      214
10      144
NaN      85
Name: count, dtype: int64